
# Quick AI Reports on Chemical Compounds  
*Session 4: Automated Report Generation *




## 🧪 Introduction

In this session, you'll see how AI can auto-generate structured reports for chemists —
short one-pagers that summarize and compare two compounds in a readable way.

**What we'll do**
- Use the OpenAI API to generate a structured report (no external databases).
- Keep outputs consistent with a clear template: title, overview, properties (as recalled by the model), similarities/differences, and a brief safety note.
- Export the report as a PDF you can download and share.

Note: Because we are not querying a database here, treat the generated content as a draft to be verified. In practice, you would pair this with RAG or database calls (like PubChem) for source-grounded facts.



## ✅ Setup (Run this once)

This installs lightweight libraries and imports everything we need.


In [ ]:

%pip -q install openai ipywidgets reportlab

import os, textwrap, datetime
import ipywidgets as widgets
from IPython.display import display, Markdown

from openai import OpenAI

# ReportLab for creating PDFs

from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.utils import simpleSplit



## 🔑 Add Your OpenAI API Key

Paste your key and click Save Key. If omitted, a placeholder string appears instead of a real model response.


In [ ]:

api_key_box = widgets.Password(description='API Key:', placeholder='sk-...', layout=widgets.Layout(width='40%'))
save_btn = widgets.Button(description='Save Key', button_style='primary')
status = widgets.HTML("")

def save_key(_):
    if api_key_box.value.strip():
        os.environ["OPENAI_API_KEY"] = api_key_box.value.strip()
        status.value = "<span style='color:green'>Key saved for this session.</span>"
    else:
        status.value = "<span style='color:#B00020'>Please paste a valid key.</span>"

save_btn.on_click(save_key)
display(widgets.HBox([api_key_box, save_btn]), status)



## 🧩 Helpers

- generate_report(compA, compB, style_hint) — ask the model for a structured one-pager.
- save_report_pdf(text, filename) — save the generated text to PDF.


In [ ]:

def _client():
    key = os.environ.get("OPENAI_API_KEY")
    return OpenAI(api_key=key) if key else None

# Defines the system role for the AI → instructs it to write short, structured reports
REPORT_SYSTEM = (
    "You are a helpful chemistry assistant. Generate a SHORT, well-structured one-page report "
    "comparing two compounds for non-expert chemists. Use clear sections and concise sentences."
)
#Builds the prompt text that we send to the model.
#always consistent: title, overview, properties, similarities/differences, notes, summary.

def _report_prompt(compA: str, compB: str, style_hint: str = "") -> str:
    today = datetime.date.today().isoformat()
    style = f" Style hint: {style_hint.strip()}" if style_hint else ""
    return f"""
Title: Quick Report - {compA} vs {compB} (Draft)
Date: {today}

Sections to include (concise, bullet-friendly):
1) Overview (1–2 sentences per compound).
2) Common/Rough Properties (what chemists typically check: use, class, notable functional groups, MW/logP if commonly known; keep general).
3) Similarities & Differences (3–6 bullets total).
4) Practical Notes (1–3 bullets: handling, typical uses, cautions).
5) Short Summary (1–2 sentences).

Rules:
- Plain, non-jargon language where possible.
- Keep to ~200–300 words total.
- If you are not sure, say so briefly (e.g., "uncertain; verify").
- Do NOT fabricate citations.{style}

Now write the report.
"""

def generate_report(compA: str, compB: str, style_hint: str = "", model: str = "gpt-4o-mini") -> str:
    client = _client()
    prompt = _report_prompt(compA, compB, style_hint)
    if client is None:
        return textwrap.dedent(f"""
        Quick Report - {compA} vs {compB} (Draft)

        Overview:
        A brief, readable comparison of {compA} and {compB}. (Demo mode - add an API key for real content.)

        Common/Rough Properties:
        • Typical properties and uses (placeholder).
        • Add MW/logP only if commonly known; otherwise mark as "verify".

        Similarities & Differences:
        • Example bullets contrasting class, effects, or structure.

        Practical Notes:
        • Basic handling notes for non-experts.

        Summary:
        • Short 1–2 sentence wrap-up. (Demo mode)
        """)
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": REPORT_SYSTEM},
                {"role": "user", "content": prompt},
            ],
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"*API error:* {e}"

def save_report_pdf(text: str, filename: str) -> str:
    path = f"{filename}"
    c = canvas.Canvas(path, pagesize=LETTER)
    width, height = LETTER

    x_margin = 0.8 * inch
    y_margin = 0.8 * inch
    max_width = width - 2*x_margin
    y = height - y_margin

    c.setFont("Times-Roman", 12)

    from reportlab.lib.utils import simpleSplit
    lines = []
    for paragraph in text.splitlines():
        wrapped = simpleSplit(paragraph, "Times-Roman", 12, max_width)
        lines.extend(wrapped if wrapped else [""])

    for line in lines:
        if y < y_margin:
            c.showPage()
            c.setFont("Times-Roman", 12)
            y = height - y_margin
        c.drawString(x_margin, y, line)
        y -= 14

    c.save()
    return path



## 💡 Demo — One-Pager (Aspirin vs Ibuprofen)

Click Generate Demo Report to produce a concise, structured one-pager using only the OpenAI API.
You can tweak style with the optional hint (e.g., "bullet-heavy, clinical tone").


In [ ]:

demo_style = widgets.Text(placeholder="Optional style hint (e.g., 'bullet-heavy, clinical tone')",
                          layout=widgets.Layout(width="60%"))
model_pick = widgets.Dropdown(
    options=[("gpt-4o-mini (fast)","gpt-4o-mini"),
             ("gpt-4o (quality)","gpt-4o"),
             ("gpt-3.5-turbo (legacy)","gpt-3.5-turbo")],
    value="gpt-4o-mini", description="Model:"
)
demo_btn = widgets.Button(description="Generate Demo Report", button_style="success")
demo_out = widgets.Output()
display(demo_style, model_pick, demo_btn, demo_out)

def run_demo(_):
    demo_out.clear_output()
    with demo_out:
        display(Markdown("Generating report for **Aspirin vs Ibuprofen** ..."))
    text = generate_report("Aspirin", "Ibuprofen", style_hint=demo_style.value, model=model_pick.value)
    pdf_path = save_report_pdf(text, "Report_Aspirin_vs_Ibuprofen.pdf")
    demo_out.clear_output()
    with demo_out:
        display(Markdown("### Report (Preview)"))
        display(Markdown(f"```{text}```"))
        display(Markdown(f"[Download PDF](sandbox:{pdf_path})"))

demo_btn.on_click(run_demo)



## ✍️ Exercise — Your Own Pair

Type any two compound names and generate a one-pager.
Keep in mind this is not database-grounded; it is a quick draft to organize thinking.

Prompting tip: Add a brief style hint to change tone/format (e.g., "more bullets, less prose").


In [ ]:

left = widgets.Text(placeholder="e.g., Caffeine", description="Left:")
right = widgets.Text(placeholder="e.g., Nicotine", description="Right:")
style = widgets.Text(placeholder="Optional style hint", description="Style:")
go_btn = widgets.Button(description="Generate Report", button_style="primary")
ex_out = widgets.Output()

display(left, right, style, model_pick, go_btn, ex_out)

def run_ex(_):
    ex_out.clear_output()
    a, b = left.value.strip(), right.value.strip()
    if not a or not b:
        with ex_out: display(Markdown("Please enter both names.")); return
    with ex_out: display(Markdown(f"Generating report for **{a} vs {b}** ..."))
    text = generate_report(a, b, style_hint=style.value, model=model_pick.value)
    safe = (a + "_vs_" + b).replace(" ", "_")
    pdf_path = save_report_pdf(text, f"Report_{safe}.pdf")
    ex_out.clear_output()
    with ex_out:
        display(Markdown("### Report (Preview)"))
        display(Markdown(f"```{text}```"))
        display(Markdown(f"[Download PDF](sandbox:{pdf_path})"))

go_btn.on_click(run_ex)



## 📘 Reflection
- Which parts of the AI report were most useful for you?
- Which statements would you verify before using (and how)?
- If you combined this with Session 2/3 retrieval, what extra reliability would you gain?
